# Kickstarter Project Success Analysis
Steve Bachmeier <br>
2018-12-12

In [5]:
import dill
df_results = dill.load(open("df_results.pkl", "rb"))

## 1 Synopsis
Several machine learning models were trained on a cleaned training set (60% of the entire dataset) and tested on a cleaned test set (20% of the entire dataset) - the results of these models are shown in the table below. 

An optimized random forest model provides a decent **10-fold cross validation mean accuracy of 71.6% while also featuring a (qualitatively) quick run-time**. 

The specific random forest model used features 100 trees, Gini impurity criterion, the maximum number of features to consider for a split equal to the square root of all of the features, and the minimum number of samples required to be at a leaf node of 25 (ie *n_estimators*=100, *criterion*="gini", *max_features*='sqrt', and *min_samples_leaf*=25).

In [4]:
df_results

,model,time_fit,time_predict,time_10_fold_CV,accuracy,acc_10_fold
0,Naive Bayes,0.101767,0.036270,1.53579,0.623551,0.624725
1,Logistic Regression,1.822522,0.009085,17.0696,0.694333,0.689984
2,K Nearest Neighbors,7.340739,101.319590,344.234,0.682739,0.675807
3,"SVM, Linear",954.375944,90.726687,None,0.677168,None
4,"SVM, RBF",1091.777649,120.427602,None,0.682908,None
5,Decision Tree,0.437127,0.014542,4.82765,0.664357,0.662544
6,Random Forest (10-fold),1.016743,0.104582,10.463,0.683248,0.680784
7,"PCA (n=2), Naive Bayes",0.024343,0.003595,0.284533,0.609355,0.608927
8,Random Forest (Optimized),5.856856,0.470576,66.9746,0.718257,0.716067
9,Logistic Regression (Optimized),6.454474,0.003956,68.8582,0.695662,0.691059


General trends were not so easy to recognize with the exception of the influence of the variable *staff_pick*; *staff_pick* is the predictor most highy correlated with *launch_state*:
* *staff_pick* - *launch_state* correlation: 25%
* 53% of projects without *staff_pick* succeed
* 89% of projects with *staff_pick* succeed

From https://www.kickstarter.com/blog/how-to-get-featured-on-kickstarter, it appears as if projects are featured when they catch the eye of the Kickstarter staff via creativity, a nice and visually appealing site, etc. ie, they are **not** just picked due to them being funded well. **Projects that are featured on Kickstarter (ie *staff_pick* = 1) are more likely to be successfully funded.**

## 2 Overview

### 2.1 Background
Having spent six years as a mechanical engineer in the silicon valley where ideas are big but funding is small, I've always been intrigued by the concept of crowd-funding. As an end user/backer, however, we want to maximize the chances that the projects we back actually successfully launch. This project uses historical data from the popular project-launching website Kickstarter to look for trends and make predictions about whether a project is likely to be successfully funded or not.

### 2.2 Data
The raw Kickstarter data (the JSON file updated at 2018-10-18) was downloaded from: https://webrobots.io/kickstarter-datasets/. It is assumed that this data is accurate and no attempt was made to verify the web scraping tools used.

Notes from raw data downloaded:

* **From April 2015 we noticed that Kickstarter started limiting how many projects user can view in a single category. This limits the amount of historic projects we can get in a single scrape run. But recent and active projects are always included.**
* **From December 2015 we modified the collection approach to go through all sub-categories instead of only top level categories. This yields more results in the datasets, but possible duplication where projects are listed in multiple categories. Also from December 2015 JSON file is in JSON streaming format. Read more about it here: https://en.wikipedia.org/wiki/JSON_Streaming**
* **We receive many question about timestamp format used in this dataset. It is unix time. Google has a lot of information about it.**
* **Files are compressed, size in area of 100mb. Uncompressed size around 600mb.**

Note that no attempt was made for this project to ensure we have the entirety of the project history. Also, due to Github size constraints, the raw dataset is not uploaded to this repository.

### 2.3 Goal
There are two potential goals of this project:

1. Analyze the raw data obtained to look for any interesting trends.

2. Build a prediction algorithm to try and predict whether future projects will successfully launch.

## 3 Analyisis
This section outlines the analysis completed. Refer to the [appendix](#Appendix) for relevant code.

### 3.1 Data preparation

Refer to appendix [A1.1 Data preparation](#A1.1) for code.

The downloaded dataset from https://webrobots.io/kickstarter-datasets/ came in JSON format; each of the 205,696 rows representing a project's details that were wrapped up in a serialized set of dictionaries and nested dictionaries. Unpacking this file was not trivial - the summary of the process is as follows:

1. Open the file with utf8 encoding and load each line to a new object.

2. The raw object includes four columns of dictionaries of which only the *data* column is relevant; extract that *data* column.

3. Convert the json file to a Pandas dataframe.

4. Unpack each dictionary with *json_normalize()*. Note that this does not unpack columns with NaN values.

5. Unpack remaining dictionary columns (those with NaN values) manually by applying the Pandas *Series()* method.

6. Concatenate the newly unpacked columns to the dataframe.

7. Drop the original json columns from the dataframe.

8. Split the dataframe into a working set (later to be the train and test set) and a validation set. We use a random 20% smapling of the entire raw dataset for the validatio set.

```
X, X_v, y, y_v = train_test_split(df_raw.drop(columns=['state']), 
                                  df_raw['state'], test_size=0.2, 
                                  random_state=101)
```

The working and validation sets now consist of 97 columns (where each columns is a different variable with one of them being the outcome).

### 3.2 Data cleaning

Refer to appendix [A1.2 Data cleaning](#A1.2) for code.

With the raw data now in a useable dataframe, we can clean it up for machine learning use.

#### 3.2.1 Clean up columns

The first step is to drop clearly useless variables. This demands some amount of reasoning. For example, it is perhaps obvious that a project photo urls, creator avatar photos, and creator profile blurbs are not useful for using machine learning to make predictions. However, other variables may not be so obvious. For example, a creator's name could be used to identify the gender (which is not provided directly in the dataset) which in turn might shed some light on project success (note that for this analysis I did indeed drop the creator's name from the dataset).

One interesting variable that took special consideration is *profile_state*. Digging into it showed that there are only two unique values for *profile_state*: 'active' and 'inactive'. Further, only 11.7% of the project profiles are labeled 'active'. It is assumed that projects go 'inactive' after a certain period of latency and so cannot be used in predicting project success (ie even a successfully funded project profile may go active after some amount of time past the deadling). I decided to drop *profile_state*.

Another tricky one was *usd_type*. Frankly, I was unable to get a firm grasp on what exactly it is. There were many instances of a project country being labeled, say 'US' with it's currency being 'USD' but then *usd_type* being 'international'. Further, there were instances of empty values. Finally, the vast majority of *usd_type* is labeled international. I decided to drop it.

Other unobvious variables that I deleted include: *name*, *blurb*, *loc_state* (too granular), *location_country* (largely redundant with *loc_country* but far more granular), *currency* (the pledges are all in USD), *currency_trailing_code*, and *state_changed_at*.

Once the useless columns were dropped, I renamed *category_slug* to *category* and *state* to *launch_state*. 

Finally, I reordered the columns into a more intuitive order including putting *launch_state* first.

#### 3.2.2 Extract categories

The *category* column (initially labeled *category_slug*) included primary categories and sub-categories in the format 'primary_category/sub_category', eg 'art/painting' and 'comics/webcomics'. I simplified the *category* variable by extracting the first word, eg 'art/painting' became 'art' and 'comics/webcomics' became 'comics'.

#### 3.2.3 Drop duplicate rows
There were a fair amount of duplicate rows which are easy to remove with ```df.drop_duplicates(inplace=True)```. However, even after this, the dataset included rows that were mostly duplicates with the exception of just a few column values. In order to keep the dataset tidy (where each row is a unique observation or, in this case, project), I had to remove any rows with duplicate project IDs. I decided to, in the case of duplicate IDs, keep those with the highest *pledged* value (assuming that this was input after the other rows and so is more accurate). This was accomplished with ```df = df.sort_values('pledged', ascending=False).drop_duplicates('id').sort_index()```. These two steps resulted in no duplicate ID values and so a tidy dataset.

#### 3.2.4 Convert relevant values to datetime

At this point, *deadline* and *launched_at* contained string values that are in the unix timestamp format. These variables were converted to datetime via

```
df['deadline'] = df['deadline'].apply(datetime.utcfromtimestamp)
df['launched_at'] = df['launched_at'].apply(datetime.utcfromtimestamp)
```

#### 3.2.5 NA / Null / empty value imputation
At this point the dataframe was completely clean of any NA, Null, or empty values. This was easily checked with

```
if df.isnull().sum().sum() != 0:
    print('*** WARNING: There are null values ***')
if df.isna().sum().sum() != 0:
    print('*** WARNING: There are NA values ***')
if (df=='').sum().sum() != 0:
    print('*** WARNING: There are empty string (\'\') values ***')
```

#### 3.2.6 Clean up the outcome variabe *launch_state*

There are five values for *launch_state* (previously *state*): 'failed', 'successful', 'canceled', 'live', and 'suspended'. 

<img src="images/launch_state.jpeg" alt="launch_state count plot" style="height: 400px;"/>

For the purposes of this project, it makes sense to keep only 'failed' and 'successful' projects (since those labeled 'canceled' and 'suspended' cannot be backed to begin with and those labeled 'live' are exactly the projects we are trying to predict). We thus query only *launch_state* valuse of 'failed' and 'successful' (and type None just for completeness).

```
df.query("launch_state == 'failed' | "
         "launch_state == 'successful' | "
         "launch_state == None", inplace=True)
```

#### 3.2.7 Create dummy variables

Of the remaining variables, *category* and *country* are categorical, ie they are labels rather than numbers. For the machine learning algorithm, I needed to convert these to dummy variables:

```
category = pd.get_dummies(df['category'], drop_first=True)
country = pd.get_dummies(df['country'], drop_first=True)
```

Note that I did drop the first dummy variable to ensure the correct degrees of freedom.

#### 3.2.8 Convert remaining string variables to integers

There were still several variables with categorical binary values that could be converted to binary integers.

* *launch_state*: ['failed', 'successful'] should be [0, 1]
* *staff_pick*: [False, True] should be [0, 1]
* *spotlight*: [False, True] should be [0, 1]


I created a dictionaries to define what the string should be converted to and then mapped it to the relevant variables.

```
d_launch_state = dict(zip(['failed','successful'], range(0,2)))
launch_state = df['launch_state'].map(d_launch_state)

d_staff_pick = dict(zip([False,True], range(0,2)))
staff_pick = df['staff_pick'].map(d_staff_pick)
  
d_spotlight = dict(zip([False,True], range(0,2)))
spotlight = df['spotlight'].map(d_spotlight)
```

### 3.3 Variable reduction
At this point we have a tidy dataframe with 141,447 rows and 46 variables (most of which are dummy *country* and *category* variables). We can now look into paring down the number of variables to help reduce over-fitting of the machine learning algorithm(s).

#### 3.3.1 Zero variance
We start by checking for zero variance variables, ie those variables that do not change at all throughout the entire dataset. Note that we've already removed some of these during the data cleaning phase, but it's still a good check.

```
sel = VarianceThreshold(threshold=0.0)
sel.fit_transform(X=df.drop(columns=info_variables)).shape[1] - df.drop(columns=info_variables).shape[1]
```

There were no zero variance variables.

#### 3.3.2 Near-zero variance
It was decided not to search for or remove near-zero variance variables. One reason is that many of the dummy variables will certainly have variances near zero, eg a particularly small *country* may only appear a handfull of times in the entire data set and so the vast majority of values will be 0. Further, there is evidence that near-zero variance variables can still have a significant impace on the outcome.

#### 3.3.3 Variable - outcome correlation
Variables with a very high correlation to the outcome we are trying to predict should be given extra consideration and possibly dropped. For this analysis, I use a threshold correlation of 0.5 - a single variable is found as shown in the plot below. Note that the dashed red line is the threshold value.

<img src="images/variable_outcome_correlation.jpeg" alt="variable-outcome correlation plot" style="height: 400px;"/>

It turns out that this single variable of interest is *spotlight* which, as shown in the plot, has a perfect correlation of 1 with *launch_state*. In other words, it's a perfect predictor (which obviously seems suspicious). From https://techcrunch.com/2015/03/25/kickstarter-spotlight/, we see that spotlight happens for successfully funded projects and acts as a way to update the project timeline. It clearly does nothing in helping predict funding success; I dropped it.

The next highest correlation is *staff_pick* at 0.25, well under the threshold.

#### 3.3.4 Variable-variable correlation
Next we consider multicollinearity, ie where a variable can be predicted by other variables. One way to battle this is by ensuring all variables have a variable-variable corration beneath some threshold. For this analysis, we assume this threshold is 0.5.

The trick here is to create an upper correlation matrix with the ones diagonal removed, unstack it, sort the values in descending order, and filter by all correlation values greater than the threshold.

```
corMat_upper = corMat.where(np.triu(np.ones(corMat.shape), k=1).astype(np.bool))
corMat_upper.unstack().sort_values(kind='quicksort')[corMat_upper.unstack().sort_values(kind='quicksort') > .5]
```

The result is that a single variable pair has a correlation larger than 0.5:

<img src="images/variable_variable_correlation.jpeg" alt="variable-variable correlation plot" style="height: 400px;"/>

The variable pair in question is *US* - *GB* and has a correlation of 0.599. It does not make sense to drop a country just because it correlates with another country and so we keep both *US* and *GB* dummy variables.

### 3.4 Exploratory data analysis
It is always a good idea to do at least a bit of exploratory data analysis before diving into the machine learning aspect of a project. Creating visualizations can uncover interesting trends and also help guide further analysis.

A pairplot of all of the non-dummy variables is shown below. Unfortunately, there does not seem to be good separation between 'successful' and 'failed' projects for any single variable.

<img src="images/pair_plot.jpeg" alt="pair plot" style="height: 400px;"/>



## X Next steps

Some recommendations to improve this analysis include:

* Complete an analysis to determine the statistical influence of the 'staff_pick' variable, ie while it appears as if staff_pick = 1 results in a higher chance that a project is successfully funded, is this statistically accurate?
* Re-run the analysis without the country dummy variables.
* Re-run the analysis without the category dummy variables.
* Code the finished script so that it can accept raw data without every column. Specifically, a small enough amount of raw data may not include all of the required countries or categories currenty required to fit the model.
* Analyze the effect of including sub-categories in the analysis.

<a id='Appendix'></a>

# Appendix

<a id='A1'></a>

# A1 Code

<a id='A1.1'></a>

## A1.1 Data preparation (as of 2018-12-12)

**f_dataImport.py**

```
# -*- coding: utf-8 -*-
"""
Created on Tue Nov 27 14:06:31 2018

@author: steve
"""


#==============================================================================
#
# IMPORT LIBRARIES
#
#==============================================================================
import pandas as pd
import os
from pandas.io.json import json_normalize 
import json

#==============================================================================
#
# FUNCTION - RAW DATA IMPORT
#
#==============================================================================

def dataImport():
    '''
    This function imports the raw json data downloaded from 
    https://webrobots.io/kickstarter-datasets/ and extracts the dictionaries
    into a usable dataframe format.
    
    OUTPUTS:
        * 'df_raw.csv': raw data dataframe
    '''
    
    #-----------------------------------------
    # READ IN RAW DATA
    
    print('\n')
    print('***')
    print('NOTE:')
    print('The input file must be JSON with the same format as those'
          'downloaded from https://webrobots.io/kickstarter-datasets/')
    print('***')
        
    while True:
    
        print('\n')
        new_data = str(input('Input the new data JSON filepath you want to predict: '))
        
        if not os.path.exists(new_data):
            print('\n')
            print('The filepath \'', new_data, '\' does not exist.', sep='')
            continue
        else:
            print('\n')
            yesno = str(input(f'Confirm that \'{new_data}\' is the correct '
                              'filepath (\'y\' or \'n\'): '))
            
            if (yesno[0].lower() == "y"):
                with open(new_data, encoding="utf8") as json_file:
                     json_obj = [json.loads(line) for line in json_file]
                break
            elif (yesno[0].lower() == 'n'):
                print('\n')
                continue
            else:
                print('\n')
                print('Improper input')
                continue
    
    #-----------------------------------------
    # UNPACK RAW DATA
    
    json_obj2 = []
    # append 'data' dictionary only
    for x in range(0, len(json_obj)):
        json_obj2.append(json_obj[x]["data"])
    
    # Check
    if (len(json_obj2) - len(json_obj)) != 0:
        print('*** ERROR: Did not extract all json \'data\' entries ***')

    # ---- CONVERT TO DATAFRAME ----
    df_raw_json = pd.DataFrame(json_obj2)
    
    # ---- UNPACK DICTIONARY ENTRIES ----
    # 'category'
    df_category = json_normalize(data=df_raw_json['category'])
    df_category.columns = 'category_' + df_category.columns
    
    # 'creator'
    df_creator = json_normalize(data=df_raw_json['creator'])
    df_creator.columns = 'creator_' + df_creator.columns
    
    # 'location'
    # Must mannually unpack 'location' with pd.Series due to NaN elements
    df_location = df_raw_json['location'].apply(pd.Series)
    df_location.drop(columns=0, inplace=True)
    df_location.columns = 'location_'+df_location.columns
    df_location1 = df_location['location_urls'].apply(pd.Series)
    df_location1.drop(columns=0, inplace=True)
    df_location1.columns = 'location_urls_'+df_location1.columns
    df_location2 = df_location1['location_urls_web'].apply(pd.Series)
    df_location2.drop(columns=0, inplace=True)
    df_location2.columns = 'location_urls_web_'+df_location2.columns
    df_location3 = df_location1['location_urls_api'].apply(pd.Series)
    df_location3.drop(columns=0, inplace=True)
    df_location3.columns = 'location_urls_api_'+df_location3.columns
    # Concat 'location' dataframes
    df_location = pd.concat([df_location, df_location2, df_location3], axis=1)
    df_location.drop(columns='location_urls', inplace=True)
    
    # 'photo'
    df_photo = json_normalize(data=df_raw_json['photo'])
    df_photo.columns = 'photo_' + df_photo.columns
    
    # 'profile'
    df_profile = json_normalize(data=df_raw_json['profile'])
    df_profile.columns = 'profile_' + df_profile.columns
    
    # 'urls'
    df_urls = json_normalize(data=df_raw_json['urls'])
    df_urls.columns = 'urls_' + df_urls.columns
     
    # ---- CONCAT UNPACKED DATAFRAMES ----
    df_raw = pd.concat([df_raw_json, df_category, df_creator, df_location, 
                        df_photo, df_profile, df_urls], axis=1)
    df_raw.drop(columns=['category','creator','location','photo',
                         'profile','urls'], inplace=True)
    
    #-----------------------------------------
    # WRITE OUT
    
    df_raw.to_csv('data/df_raw.csv', sep=",")
    
    return df_raw
```

<a id='A1.2'></a>

## A1.2 Data cleaning (as of 2018-12-12)

**f_cleanData.py**

```
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 10 13:04:33 2018

@author: steve
"""

#==============================================================================
#
# IMPORT LIBRARIES
#
#==============================================================================

import pandas as pd
from datetime import datetime

#==============================================================================
#
# FUNCTION - CLEAN DATA
#
#==============================================================================
def cleanData(df):
    '''
    This function cleans downloaded raw data for the Kickstarter success 
    prediction project. The input dataframe must be imported and the json 
    extracted using '00 - Data Import.py'. There should be either 95 or 96
    columns (the 'state' column is optional) and they must be labeled exactly
    as defined in '00 - Data Import.py'.
    '''
    
    #-----------------------------------------
    # ADD EMPTY STATE COLUMN IF NECESSARY
    if 'state' not in df.columns:
        df['state'] = None
    
    #-----------------------------------------
    # COLUMN CLEANUP
    
    drop_vars = ['photo_1024x576', 'photo_1536x864', 'photo_ed', 'photo_full', 
                 'photo_key', 'photo_little', 'photo_med', 'photo_small', 
                 'photo_thumb', 'slug', 'urls_api.message_creator', 'urls_api.star', 
                 'urls_web.message_creator', 'urls_web.project', 'urls_web.rewards', 
                 'source_url', 'creator_avatar.medium', 'creator_avatar.small', 
                 'creator_avatar.thumb', 'creator_chosen_currency', 'creator_id', 
                 'creator_name', 'creator_slug', 'creator_urls.api.user',
                 'creator_urls.web.user', 'location_id', 'location_name', 
                 'location_slug', 'location_short_name', 'location_displayable_name', 
                 'location_localized_name', 'location_type', 'location_is_root', 
                 'location_urls_web_discover', 'location_urls_web_location', 
                 'location_urls_api_nearby_projects', 'category_color', 
                 'category_id', 'category_urls.web.discover', 
                 'profile_background_color', 
                 'profile_background_image_attributes.id', 
                 'profile_background_image_attributes.image_urls.baseball_card', 
                 'profile_background_image_attributes.image_urls.default',
                 'profile_background_image_opacity', 'profile_blurb', 
                 'profile_feature_image_attributes.id', 
                 'profile_feature_image_attributes.image_urls.baseball_card',
                 'profile_feature_image_attributes.image_urls.default', 'profile_id',
                 'profile_link_background_color', 'profile_link_text', 
                 'profile_link_text_color', 'profile_link_url', 'profile_name', 
                 'profile_project_id', 'profile_should_show_feature_image_section', 
                 'profile_show_feature_image', 'profile_state', 
                 'profile_state_changed_at', 'profile_text_color', 'currency_symbol',
                 'static_usd_rate','converted_pledged_amount','fx_rate',
                 'current_currency', 'usd_pledged', 'is_starrable', 'friends', 
                 'is_backing', 'is_starred', 'permissions', 'name', 'blurb',
                 'location_state', 'location_country', 'currency', 
                 'currency_trailing_code', 'state_changed_at', 'category_parent_id', 
                 'category_position', 'category_name', 'category_id', 
                 'creator_is_registered', 'disable_communication', 'created_at', 
                 'usd_type']

    df.drop(columns=drop_vars, inplace=True)
    
    # Rename columns
    df.rename(columns={'category_slug':'category', 'state':'launch_state'}, inplace=True)
    
    # Rearrange columns
    df = df[['launch_state', 'id', 'category', 'goal', 'backers_count', 
             'pledged', 'country','deadline', 'launched_at', 
             'staff_pick', 'spotlight']]

    #-----------------------------------------
    # EXTRACT CATEGORIES
    df['category'] = [i.split('/')[0] for i in df['category']]
    
    #-----------------------------------------
    # REMOVE DUPLICATES
    df.drop_duplicates(inplace=True)
    # For duplicate IDs leftover, remove the lesser pledged row
    df = df.sort_values('pledged', ascending=False).drop_duplicates('id').sort_index()
    
    # Check
    if (len(df) - len(df["id"])) != 0:
        print('*** WARNING: There are ',
              len(df) - len(df["id"]), 
              ' duplicate IDs ***', sep='')
    
    #-----------------------------------------
    # CONVERT DATETIMES
    df['deadline'] = df['deadline'].apply(datetime.utcfromtimestamp)
    df['launched_at'] = df['launched_at'].apply(datetime.utcfromtimestamp)
    
    #-----------------------------------------
    # NA IMPUTATION 
    # Checks
    if df.isnull().sum().sum() != 0:
        print('*** WARNING: There are null values ***')
    if df.isna().sum().sum() != 0:
        print('*** WARNING: There are NA values ***')
    if (df=='').sum().sum() != 0:
        print('*** WARNING: There are empty string (\'\') values ***')
    
    #-----------------------------------------
    # CLEAN UP 'launch_state'
    df.query("launch_state == 'failed' | "
             "launch_state == 'successful' | "
             "launch_state == None", inplace=True)
    
    #-----------------------------------------
    # CONVERT CATEGORICAL VARIABLES TO DUMMY VARIABLES 
    category = pd.get_dummies(df['category'], drop_first=True)
    country = pd.get_dummies(df['country'], drop_first=True)
    d_launch_state = dict(zip(['failed','successful'], range(0,2)))
    launch_state = df['launch_state'].map(d_launch_state)
    
    # Check
    if (df[df['launch_state'] == 'successful'].shape[0] - launch_state.sum() != 0):
        print('*** WARNING: Some launch_states did not map to 0/1 ***')
    
    # Drop the categorical launch_state column 
    # (keep 'category' and 'country' for  visualization)
    df.drop(['launch_state'],axis=1,inplace=True)
    
    # Add the new dummy variable launch_state column and move it to column index 0 
    # and country to column index 3
    df = pd.concat([launch_state, df], axis=1)
    df = df[['launch_state', 'id', 'category', 'country', 'goal', 'backers_count', 
             'pledged','deadline', 'launched_at', 'staff_pick', 'spotlight']]
    
    # Add the dummy variable country and category columns
    df = pd.concat([df, category, country], axis=1)
    
    # Checks
    if (df.isnull().sum().sum() != 0):
        print('*** WARNING: Null values introduced with dummy variables ***')
    if (df.isna().sum().sum() != 0):
        print('*** WARNING: NA values introduced with dummy variables ***')
    if (df=='').sum().sum() != 0:
        print('*** WARNING: Empty string (\'\') values introduced with dummy variables ***')
    
    #-----------------------------------------
    # FINAL CLEANUP
    # pledged_ratio
    pledged_ratio = df['pledged'] / df['goal']
    df.insert(loc=df.columns.get_loc("pledged"), column='pledged_ratio', 
              value=pledged_ratio)
    df.drop(columns='pledged', inplace=True)
    
    # datetime columns
    funding_days = (df['deadline'] - df['launched_at']).dt.days
    df.insert(loc=df.columns.get_loc("deadline"), column='funding_days', 
              value=funding_days)
    df.drop(columns='deadline', inplace=True)
    
    # ---- MOVE 'LAUNCHED_AT' ----
    launched_at = df['launched_at']
    df.drop(columns='launched_at', inplace=True)
    df.insert(loc=2, column='launched_at', value=launched_at)
    
    # ---- CONVERT 'STAFF_PICK' AND 'SPOTLIGHT' TO DUMMIES ----
    d_staff_pick = dict(zip([False,True], range(0,2)))
    staff_pick = df['staff_pick'].map(d_staff_pick)
    
    # Check
    if (df[df['staff_pick'] == True].shape[0] - staff_pick.sum()) != 0:
        print('*** WARNING: \'staff_pick\' not mapped to 0/1 properly ***')
        
    d_spotlight = dict(zip([False,True], range(0,2)))
    spotlight = df['spotlight'].map(d_spotlight)
    
    # Check
    if (df[df['spotlight'] == True].shape[0] - spotlight.sum()) != 0:
        print('*** WARNING: \'spotlight\' not mapped to 0/1 properly ***')
        
    df.drop(['staff_pick','spotlight'],axis=1,inplace=True)
    
    df.insert(loc=df.columns.get_loc("comics"), column='staff_pick', value=staff_pick)
    df.insert(loc=df.columns.get_loc("comics"), column='spotlight', value=spotlight)
    
    #-----------------------------------------
    # VARIABLE REDUCTION
    df.drop(columns='spotlight', inplace=True)
    
    # ---- NULL/NA/EMPTY CHECKS ----
    if (df.isnull().sum().sum() != 0):
        print('*** WARNING: Null values introduced with \'staff_pick\' and \'spotlight\' dummy variables ***')
    if (df.isna().sum().sum() != 0):
        print('*** WARNING: NA values introduced with \'staff_pick\' and \'spotlight\' dummy variables ***')
    if (df=='').sum().sum() != 0:
        print('*** WARNING: Empty string (\'\') values introduced with \'staff_pick\' and \'spotlight\' dummy variables ***')
    
    #-----------------------------------------
    # WRITE OUT
    df.to_csv('df_clean.csv', sep=",")
    
    return df
```